In [1]:
import numpy as np
import numpy.ma as ma
import pandas as pd
import tensorflow as tf
# from tensorflow import keras
from numpy import genfromtxt
import joblib
import requests
import json

load data

In [2]:
user_vecs = genfromtxt('./data/user_vecs.csv', delimiter=',')
item_vecs = genfromtxt('./data/item_vecs.csv', delimiter=',')
item_embeddings = genfromtxt('./data/item_embeddings.csv', delimiter=',')

In [3]:
vms = item_embeddings

load scaler

In [4]:
scalerUser = joblib.load('scalerUser.save')
scalerItem = joblib.load('scalerItem.save')
scalerTarget = joblib.load('scalerTarget.save')

Create dictionary for user_feature

In [5]:
user_dict = {}
for i in range(len(user_vecs)):
    user_dict[int(user_vecs[i][0])] = user_vecs[i]

In [6]:
user_dict[2]

array([ 2.  , 22.  ,  4.  ,  3.95,  4.25,  0.  ,  0.  ,  4.  ,  4.12,
        4.  ,  4.04,  0.  ,  3.  ,  4.  ,  0.  ,  3.88,  3.89])

# load model for local testing

Below, we also launch the user embedding model through docker container

In [9]:
# model = tf.keras.models.load_model('user_embedding_model')

2023-06-29 17:29:20.894958: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [37]:
user_dict.keys()

dict_keys([2, 3, 4, 7, 9, 10, 12, 13, 15, 16, 17, 18, 20, 21, 22, 23, 24, 25, 28, 29, 30, 34, 36, 41, 42, 45, 47, 48, 49, 50, 51, 52, 55, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 72, 73, 74, 75, 76, 77, 78, 80, 82, 83, 86, 87, 88, 89, 91, 92, 95, 97, 98, 100, 103, 104, 105, 106, 108, 110, 111, 112, 114, 116, 119, 122, 123, 124, 125, 127, 129, 131, 132, 135, 137, 138, 139, 140, 141, 143, 144, 146, 148, 152, 153, 154, 156, 158, 159, 160, 161, 165, 166, 167, 168, 169, 172, 175, 177, 178, 180, 182, 184, 185, 186, 187, 189, 190, 193, 194, 195, 196, 198, 199, 200, 203, 204, 205, 209, 210, 211, 212, 213, 215, 218, 219, 220, 221, 222, 223, 226, 227, 228, 230, 231, 232, 233, 234, 237, 238, 239, 241, 246, 247, 248, 249, 250, 251, 252, 253, 254, 256, 257, 258, 259, 260, 261, 263, 264, 271, 272, 274, 275, 278, 279, 280, 281, 282, 285, 286, 287, 288, 291, 292, 293, 295, 296, 298, 300, 301, 303, 305, 306, 307, 308, 309, 310, 311, 312, 313, 316, 317, 318, 319, 320, 322, 323, 325, 326, 327, 328, 33

Generate user embedding

In [38]:
uid = 3

In [39]:
user_vec = user_dict[uid].reshape(1,17)

In [40]:
scaled_user_vec = scalerUser.transform(user_vec)
# vu = model.predict(scaled_user_vec[:,3:])

In [41]:
data = json.dumps({"instances":scaled_user_vec[:,3:].tolist()})

In [42]:
import requests
import json
headers = {"content-type": "application/json"}
json_response = requests.post('http://localhost:8501/v1/models/user_embedding_model:predict', data=data, headers=headers)
predictions = json.loads(json_response.text)['predictions']

In [43]:
vu = np.array(predictions)

In [44]:
vu

array([[-0.1435131 ,  0.15254034, -0.17324552,  0.05825283, -0.21046445,
        -0.05286913, -0.05887726,  0.11435975,  0.08815061,  0.23393752,
        -0.15388033,  0.03621101, -0.25165731, -0.10379627,  0.1871776 ,
        -0.05175064,  0.29418603,  0.13672602, -0.03525276, -0.18706848,
         0.20332037, -0.00785702, -0.23737656,  0.09011389, -0.0702479 ,
        -0.20614502,  0.30533364, -0.16856277,  0.20556332, -0.20866966,
        -0.12087867,  0.3780967 ]])

In [45]:
y_p = np.dot(vms, vu.T)
y_pu = scalerTarget.inverse_transform(y_p)

In [46]:
sorted_index = np.argsort(-y_pu,axis=0).reshape(-1).tolist()  #negate to get largest rating first

In [53]:
item_vecs[:,0]

array([  4054.,   4069.,   4148.,   4149.,   4153.,   4161.,   4167.,
         4223.,   4228.,   4232.,   4238.,   4239.,   4246.,   4247.,
         4254.,   4270.,   4299.,   4306.,   4308.,   4310.,   4340.,
         4343.,   4344.,   4366.,   4367.,   4368.,   4369.,   4370.,
         4386.,   4388.,   4446.,   4447.,   4448.,   4638.,   4639.,
         4641.,   4643.,   4700.,   4701.,   4718.,   4720.,   4728.,
         4734.,   4776.,   4816.,   4823.,   4848.,   4865.,   4873.,
         4874.,   4876.,   4878.,   4881.,   4886.,   4887.,   4890.,
         4896.,   4901.,   4902.,   4903.,   4958.,   4963.,   4967.,
         4973.,   4974.,   4975.,   4979.,   4992.,   4993.,   4994.,
         4995.,   5010.,   5013.,   5014.,   5015.,   5025.,   5026.,
         5064.,   5065.,   5066.,   5072.,   5108.,   5110.,   5128.,
         5135.,   5152.,   5171.,   5218.,   5219.,   5220.,   5254.,
         5265.,   5266.,   5267.,   5283.,   5294.,   5296.,   5299.,
         5313.,   53

Return the top 10 movie IDs

In [47]:
item_vecs[sorted_index[:10],0].astype(int)

array([ 6283, 60684, 27801,  6874, 32587,  6773,  8370, 69481,  5072,
        7438])

In [7]:
def retrieve(uid):
    #compute user embedding
    user_vec = user_dict[uid].reshape(1,17)
    scaled_user_vec = scalerUser.transform(user_vec)
    vu = model.predict(scaled_user_vec[:,3:])
    
    #compute dot product between user embedding and item embedding
    y_p = np.dot(vms, vu.T)
    y_pu = scalerTarget.inverse_transform(y_p)
    
    #Sort the result
    sorted_index = np.argsort(-y_pu,axis=0).reshape(-1).tolist()  #negate to get largest rating first
    
    #return the top 10 movies
    result = item_vecs[sorted_index[:10],0].astype(int)
    
    return result
    

deploy a fastapi

In [8]:
import os
import io
import uvicorn
import numpy as np
import json
import nest_asyncio
from enum import Enum
from fastapi import FastAPI, UploadFile, File, HTTPException
from fastapi.responses import StreamingResponse

In [9]:
from pydantic import BaseModel

In [10]:
app = FastAPI(title='Deploying a ML Model with FastAPI')

In [11]:
class User(BaseModel):
    uid: int

In [16]:
@app.get("/")
def home():
    return "Congratulations! Your API is working as expected. Now head over to http://localhost:8000/docs."

@app.post("/predict") 
def prediction(user: User):

    payload = user.uid
    print('payload is:',payload)
    response = retrieve(payload).tolist()
    print("response is:", response)
    
    return {"Recommended":response}

In [ ]:
# Allows the server to be run in this interactive environment
nest_asyncio.apply()

# Host depends on the setup you selected (docker or virtual env)
host = "0.0.0.0" if os.getenv("DOCKER-SETUP") else "127.0.0.1"

# Spin up the server!    
uvicorn.run(app, host=host, port=8000)

INFO:     Started server process [79761]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:51462 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:51462 - "GET /openapi.json HTTP/1.1" 200 OK
payload is: 2
response is: [150548, 114935, 105504, 80906, 115149, 81562, 27846, 122920, 104841, 96610]
INFO:     127.0.0.1:51464 - "POST /predict HTTP/1.1" 200 OK
